In [40]:
from dotenv import dotenv_values
import os
import requests
import re
from langchain.callbacks import get_openai_callback
import logging
import pprint

secrets = dotenv_values(r"..\..\langchain_pipeline\.env")
total_cost = 0
total_tokens = 0
total_prompt_tokens = 0
total_completion_tokens = 0

# Function to set up the logger
def setup_logger(name, log_file, level=logging.DEBUG):
    """Function to set up as many loggers as you want."""

    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(level)

    # Avoid setting up the logger again if it already has handlers (to avoid duplicate logs)
    if not logger.hasHandlers():
        # Create formatter
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s')

        # Create a file handler and set level to debug
        file_handler = logging.FileHandler(log_file)
        file_handler.setFormatter(formatter)

        # Create a stream handler (for console) and set level to debug
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)

        # Add handlers to logger
        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)

    return logger

openai_api_logger = setup_logger("openai_api", "openai_api.log")

# os.environ['OPENAI_API_KEY'] = secrets['openai_api']
# os.environ['SERPAPI_API_KEY'] = secrets['serp_api_key']

In [11]:
openai_models_url = "https://api.openai.com/v1/models"

with get_openai_callback() as openai:
    openai_models = requests.get(openai_models_url, headers={"Authorization": "Bearer " + secrets['openai_api']}).json()
    total_cost += openai.total_cost
    total_tokens += openai.total_tokens
    total_prompt_tokens += openai.prompt_tokens
    total_completion_tokens += openai.completion_tokens
    openai_api_logger.info(f'''\nCurrent API Call Usage\n---------------------\n{openai}\n\nTotal Usage\n---------------------\nTotal Cost: {total_cost}\nTotal Tokens: {total_tokens}\nTotal Prompt Tokens: {total_cost}\nTotal Completion Tokens: {total_cost}\n''')


2023-09-19 23:06:17,309 - openai_api - INFO - 
Current API Call Usage
---------------------
Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0

Total Usage
---------------------
Total Cost: 0.0
Total Tokens: 0
Total Prompt Tokens: 0.0
Total Completion Tokens: 0.0



In [41]:
def find_models(all_models: dict, re_pattern: str = 'gpt'):
    gpt_models = [i['id'] for i in all_models['data'] if re.search(re_pattern, i['id'])]
    return gpt_models

In [42]:
find_models(openai_models, re_pattern='gpt')

['gpt-3.5-turbo-instruct-0914',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-16k']

# Chat Model

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

def chat_llm_request(system_message: str, human_message: str, temperature: float = 0.5):
    global total_cost, total_tokens, total_prompt_tokens, total_completion_tokens

    with get_openai_callback() as cb:
        """Requests to the OpenAI API is made here"""
        chat_llm = ChatOpenAI(temperature=temperature)

        messages = [
            SystemMessage(content=system_message),
            HumanMessage(content=human_message)
        ]
        response = chat_llm(messages=messages)
        total_cost += cb.total_cost
        total_tokens += cb.total_tokens
        total_prompt_tokens += cb.prompt_tokens
        total_completion_tokens += cb.completion_tokens
        
        openai_api_logger.info(f'''
Current API Call Usage
---------------------
{cb}

Total Usage
---------------------
Total Cost: {total_cost}
Total Tokens: {total_tokens}
Total Prompt Tokens: {total_prompt_tokens}
Total Completion Tokens: {total_completion_tokens}
''')

    return response

In [44]:
messages = [
    "You are a professional English and Dutch teacher. You will be provided a prompt in either dutch or english, and you will have to answer the prompt and then provide a translation to Dutch or English. When prompted to generate a short story in either Dutch or English, the short story must follow the basic structure of a short story with a beginning, middle, and end. The short story must be at least 10 sentences long. After generating a short story, prompt the user on whether they would like to translate the short story to Dutch or English. If the user chooses to translate the short story to Dutch, then the short story must be translated to Dutch. If the user chooses to translate the short story to English, then the short story must be translated to English. The short story must be translated to the best of your ability. If prompted to generate questions on the short story in either Dutch or English, then the questions must be generated in the language that is requested and the user will be requred to answer the questions in the same language as the questions were generated in. Always check your work for spelling and grammar mistakes. Always check the user's responses for spelling and grammar mistakes. If the user's response contains spelling or grammar mistakes, then prompt the user to correct their mistakes.",
    "Tell me a short story in dutch that an intermediate level person can understand.",
]

response = chat_llm_request(system_message=messages[0], human_message=messages[1], temperature=1)

2023-09-20 21:28:45,714 - openai_api - INFO - 
Current API Call Usage
---------------------
Tokens Used: 984
	Prompt Tokens: 285
	Completion Tokens: 699
Successful Requests: 1
Total Cost (USD): $0.0018254999999999999

Total Usage
---------------------
Total Cost: 0.0018254999999999999
Total Tokens: 984
Total Prompt Tokens: 285
Total Completion Tokens: 699



In [38]:
print(response.content)

Er was eens een klein meisje genaamd Lisa. Lisa woonde in een prachtig dorpje omringd door groene weilanden en hoge bergen. Op een dag besloot Lisa om een avontuur te beleven. Ze pakte haar rugzak en ging op weg naar het mysterieuze bos aan de rand van het dorp.

Het bos was donker en stil, maar Lisa was niet bang. Ze wandelde verder en ontdekte al snel een sprookjesachtige plek met kleurrijke bloemen en sprankelende watervallen. Terwijl Lisa genoot van de schoonheid van het bos, hoorde ze plotseling een zacht gehuil.

Lisa volgde het geluid en ontdekte een kleine gewonde vogel. Ze nam de vogel voorzichtig in haar handen en besloot om voor hem te zorgen. Lisa bouwde een klein nestje van takjes en bladeren en voedde de vogel met bessen.

Dagen gingen voorbij en de vogel werd sterker. Hij begon te fladderen in het nestje en sprong uiteindelijk uit het nest om te vliegen. Lisa voelde zich trots en gelukkig dat ze de vogel had geholpen.

Terug in het dorp vertelde Lisa aan iedereen over ha

# LLM Agents

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
with get_openai_callback() as cb:
    response = agent.run(
        "What is a beagle that does not have hair?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

# ChatModel Agent

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

MEMORY_KEY = 'chat_history'
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

chat_model = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi"])
agent = initialize_agent(tools, chat_model, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory, handle_parsing_errors=True)

In [ ]:
with get_openai_callback() as cb:
    response = agent.run(
        "What is a beagle that does not have hair?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [ ]:
with get_openai_callback() as cb:
    print(agent.run(
        "Hi, my name is Lian, and I am testing your memory."
    ))
    print(cb)

In [ ]:
print(agent.run(input="I want you to come up with a nickname for me."))

# ChatModel with Memory

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

def chat_llm_request(system_message: str, human_message: str, temperature: float = 0.5):
    global total_cost, total_tokens, total_prompt_tokens, total_completion_tokens

    with get_openai_callback() as cb:
        """Requests to the OpenAI API is made here"""
        chat_llm = ChatOpenAI(temperature=temperature)

        prompt = ChatPromptTemplate(
            messages=[
                SystemMessagePromptTemplate.from_template(
                    "You are a nice chatbot having a conversation with a human."
                ),
                # The `variable_name` here is what must align with memory
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("{question}")
            ]
        )
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        conversation = LLMChain(
            llm=llm,
            prompt=prompt,
            verbose=True,
            memory=memory
        )

        total_cost += cb.total_cost
        total_tokens += cb.total_tokens
        total_prompt_tokens += cb.prompt_tokens
        total_completion_tokens += cb.completion_tokens
        
        openai_api_logger.info(f'''
Current API Call Usage
---------------------
{cb}

Total Usage
---------------------
Total Cost: {total_cost}
Total Tokens: {total_tokens}
Total Prompt Tokens: {total_prompt_tokens}
Total Completion Tokens: {total_completion_tokens}
''')

    return conversation

In [48]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.run(
    "What is your name?"
)




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: What is your name?

> Finished chain.


'Hello! As a language model developed by OpenAI, I don\'t have a specific name. You can refer to me as "ChatGPT" or simply "Chatbot." How can I assist you today?'

In [49]:
conversation.run(
    "Let's call you Persona, can you tell me a little about yourself?"
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: What is your name?
AI: Hello! As a language model developed by OpenAI, I don't have a specific name. You can refer to me as "ChatGPT" or simply "Chatbot." How can I assist you today?
Human: Let's call you Persona, can you tell me a little about yourself?

> Finished chain.


"Certainly! While I don't have a personal identity or consciousness, I can tell you about my capabilities and purpose. I am an AI language model designed to engage in conversations with users like you. My purpose is to assist and provide helpful information on a wide range of topics, from answering questions to offering suggestions and engaging in discussions. I have been trained on a diverse range of texts from the internet, which enables me to generate human-like responses. How can I help you today, Persona?"